In [215]:
import socket
import random
import numpy as np

In [226]:
def buildPacket(sync = 0, ack = 0, fin = 0, seqNum = 0, ackNum = 0, payload = 0, divisor = "000000000"):
    flags = f"000{ack}00{sync}{fin}"
    stringPacket = flags + f"{seqNum:032b}" + f"{ackNum:032b}" + f"{payload:01024b}"
    CRC = ""
    if int(divisor) > 0:
        CRC = getCRC(stringPacket, divisor)
    else:
        CRC = "00000000"

    stringPacket += CRC
    
    listPacket = np.array([val for val in stringPacket]).reshape(int(np.ceil(len(stringPacket)/8)),8)
    intPacket = [int("".join(listByte), base = 2) for listByte in listPacket]

    return intPacket

In [217]:
def getCRC(binCode, divisor):
    payload = list(binCode) + ([0]*(len(divisor)-1))
    payload = [int(n) for n in payload]
    divisor = [int(n) for n in list(divisor)]
    remainder = []
    pointer = 0

    while pointer <= len(payload) - len(divisor):
        #Extract slice to subtract
        val = payload[pointer: pointer + len(divisor)]
        
        #Subtract paired values
        operations = zip(val, divisor)
        remainder = [abs(op[0] - op[1]) for op in operations]
        
        #Modify payload values, set pointer to index
        for i in range(len(divisor)):
            payload[pointer + i] = remainder[i]
        
        #Move pointer to next non-zero value
        pointer = payload.index(1)
    return "".join([str(val) for val in remainder])[1:]

In [218]:
def reconstructHeader(packet, low, high):
    headerVal = int("".join([f"{packByte:08b}" for packByte in packet[low:high]]), base = 2)
    return headerVal

In [249]:
def separateData(packet):
    stringPacket = [f"{packByte:08b}" for packByte in packet]
    flags = stringPacket[0]
    seqNum = int("".join(stringPacket[1:5]), base = 2)
    ackNum = int("".join(stringPacket[5:9]), base = 2)
    payload = "".join(stringPacket[9:-1])
    return (flags, seqNum, ackNum, payload)


In [250]:
def decipherMessage(message):
    msgBytes = np.array([char for char in message]).reshape(int(len(message)/8), 8)
    letters = [chr(int("".join(msgByte), base = 2)) for msgByte in msgBytes if chr(int("".join(msgByte), base = 2)) != "\x00"]
    return "".join(letters)

In [251]:
def checkCRC(binCode, divisor):
    stringPacket = [f"{packByte:08b}" for packByte in binCode]
    payload = []
    for byteSection in stringPacket:
        payload += byteSection
    payload = [int(n) for n in payload]
    divisor = [int(n) for n in list(divisor)]
    remainder = []
    pointer = 0

    while pointer <= len(payload) - len(divisor):
        #Extract slice to subtract
        val = payload[pointer: pointer + len(divisor)]
        
        #Subtract paired values
        operations = zip(val, divisor)
        remainder = [abs(op[0] - op[1]) for op in operations]
        
        #Modify payload values, set pointer to index
        for i in range(len(divisor)):
            payload[pointer + i] = remainder[i]
        
        #Move pointer to next non-zero value
        if 1 not in payload:
            return 0
        pointer = payload.index(1)

    return int("".join([str(val) for val in remainder])[1:])

In [253]:
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind(("localhost", 8001))
    s.listen()
    while True:
        cnt = 0
        conn, adr = s.accept()
        with conn:
            print(f"Connected to {adr}")

            seqNum = 0
            ackNum = 1
            otherDivisor = "100000110"
            totalPayload = ""

            # Receive Sync Request
            syncReq = conn.recv(2048)
            sync = int(f"{syncReq[0]:08b}"[6])

            if sync:
                print("Client wants to synchronize")
            else:
                conn.close()
                break

            # Accept Sync Request
            ack = buildPacket(sync=1, ack=1, ackNum=ackNum, seqNum=seqNum)
            seqNum += 1
            conn.send(bytes(ack))

            # Receive Sync Ack
            syncAck = int(f"{conn.recv(2048)[0]:08b}"[3])
            if (syncAck):
                print("Ready to receive data")

            else:
                conn.close()
                break            



            while True:
                failRate = random.random()
                
                # Receive Data
                print("waiting for data")
                packet = conn.recv(2048)
                print("processing data")
                clFlags, clSeqNum, clAckNum, payloadPiece = separateData(packet)

                if failRate < 0.9:
                    print("Packet Error, try to request packet")
                    stringPacket = [f"{packByte:08b}" for packByte in packet]
                    bit1 = int(stringPacket[-2])
                    bit2 = int(stringPacket[-1])
                    stringPacket[-1] = stringPacket[-1][:-2] + str(bit1^1) + str(bit2^1)
                    packet = [int("".join(listByte), base = 2) for listByte in stringPacket]




                fin = int(clFlags[-1])
                crcCheck = checkCRC(packet, otherDivisor)
                print(crcCheck)
                
                if fin:
                    print("Client is finished sending data")
                    conn.close()
                    break
                
                if crcCheck == 0:
                    ackNum += len(packet)
                    totalPayload += payloadPiece
                
                
                # Send Acknowledgement
                print("ack", ackNum)
                ack = buildPacket(ack=1, ackNum=ackNum, seqNum=seqNum)
                conn.send(bytes(ack))

                # # Data
                # msg_bit = bytearray(conn.recv(2048))

                # if not msg_bit:
                #     break

                print(f"cnt: {cnt} >>> {payloadPiece}")
                cnt += 1

                # cnt += 1

                # if failRate < .5:
                #     print(msg_bit[-1])

                #     lBin = int(bin(msg_bit[-1])[2:]) ^ 11
                #     print(lBin)

                #     lBin = int('0b' + str(lBin), base=2)
                #     print(lBin)

                #     # msg_bit = msg_bit[:-2] + b'FF'
                #     msg_bit.pop()
                #     msg_bit.append(lBin)
                #     print("Flipped bit", msg_bit)
                #     # print(altm.decode())

                # # conn.send(bytes("ack".encode("ascii", errors="ignore")))
                # conn.send(msg_bit)

                # msg += msg_bit
                # if cnt > 3:
                # break

            # conn.sendall(msg)
            # with open("recImage.jpg", "wb") as jpg:
                # jpg.write(msg)

        print("Displaying data")
        print(decipherMessage(totalPayload))
        break


Connected to ('127.0.0.1', 8081)
Client wants to synchronize
Ready to receive data
waiting for data
processing data
Packet Error, try to request packet
100011
ack 1
cnt: 0 >>> 00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [161]:
s.close()

In [22]:
# Generate random number in the range of 0 to 10
while True:
    num = random.randint(0,10)

    msg, adr = s.recvfrom(2048)
    print(f"[{cnt}]: {adr} >> {msg}")
    cnt += 1

    # If rand is less is than 2, the packet got errors by inverting the last two bits
    if num < 2:
        msg = msg[:-2] + b'FF'
    
    # Otherwise, the server responds
    s.sendto(msg, adr)
    if num < 2:
        print(f"Message with errors sent to {adr}")
    else:
        print(f"Message sent back normally {adr}")

    

KeyboardInterrupt: 

In [83]:
s.close()

In [7]:
# Receive the client packet along with the address it is coming from
connection, addr = s.accept()  

In [8]:
# If rand is less is than 2, the packet got errors by inverting the last two bits
if num < 2:
    # Inver last 2 bits
    pass

In [ ]:
# Otherwise, the server responds
